In [73]:
# 셀레니늄 모듈 설치
# 별도 터미널에서 진행
# https://selenium-python.readthedocs.io/ 셀레니움 파이썬에 관련된 공홈
# !conda install selenium -y

In [93]:
# 셀레니움 모듈 가져오기(웹드라이버만)
# 액션이 있는 난이도 있는 웹에서 긁을라고 셀레니윰 뜬다.
from selenium import webdriver as wd
# 시간
import time

In [97]:
# 웹드라이버 프로그램 준비
# 통상 크롬을 사용한다면, 개발 PC에 설치된 크롬 버전과 동일한 웹드라이버 획득
# 버전 107.0.5304.107(공식 빌드) (64비트)
# https://chromedriver.chromium.org/downloads 여기접속
# 웹드라이버 가동
driver = wd.Chrome('./chromedriver.exe')

In [95]:
# 접속
# 사이트가 로딩되는 속도가 천차 만별이다.
# 로딩이 완료되었는지 검사하는 루틴이 필요
# 귀찮으면 명시적으로 특정 시간을 대기하면 된다.
driver.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')

# 강제로 10초 대기(명시적 대기)
time.sleep(1*3)

In [12]:
# 시도를 특정한다
# 주관적으로 수행했는데 반복성이 안보여서, 기계적으로 생성
# 셀레니움 css selector가 간혹 잘 안될떄도 있어서 잘 되는 표현으로 사용
# 길어도 특정한놈 1개만 있는거 잡으면 그 앞으로 지워도딤
'''
div.loca_step1_cont > ul > li:nth-child(1) > a
'''
# 서울클릭
css_sel = 'div.loca_step1_cont > ul > li:nth-child(1) > a'
si_do_a_tag = driver.find_element_by_css_selector(css_sel)
si_do_a_tag.click()

time.sleep(1*3)

In [13]:
# 시군구 클릭
# 서울 > 중구 클릭 실행 => 전체가 있다 => 전체 클릭으로 변경
css_sel2 = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(css_sel2).click()

time.sleep(1*3)

In [83]:
# 최종 목적 페이지까지 로드되면, 셀레니늄으로 처리할지, BS4로 처리할지 결정 필요
len(driver.find_elements_by_css_selector('#mCSB_3_container > ul > li'))

598

In [14]:
#BS4
from bs4 import BeautifulSoup
src = driver.page_source  # 현재 페이지의 HTML 소스
soup = BeautifulSoup(src,'html5lib')

In [15]:
len( soup.select('#mCSB_3_container > ul > li'))
# .quickSearchResultBoxSidoGugun > li 이렇게 해도됨 li 부모 특정한거임(독립적이니까 하나밖에없으니까)

598

In [20]:
# 추출정보 : 전국구의 지점명, 위도, 경도, 주소
starbucks_store_infos = list()
for li in soup.select('.quickSearchResultBoxSidoGugun > li'):
    starbucks_store_infos.append({
        'name': li.get('data-name') , # 지점명, 속성값 추출 => 요소.get('속성이름')
        'lat' : float( li.get('data-lat') ) ,   # 위도 -> float()
        'long': float( li.get('data-long')) ,   # 경도 -> float() 
        'code': li.get('data-code') , # 지점코드
        'storecd': li.get('data-storecd') ,                 # 관리코드(?)
        'addr': li.p.text.strip()[:len('1522-3232')*-1] ,   # 전화번호가 모두 동일하다
        'spec': li.i.get('class')[0][len('pin_'):]  # class는 속성값으로 뽑으면 리스트로 나온다(특징), pin_ 제거    
    })
    #break
len( starbucks_store_infos ),starbucks_store_infos[0]

(598,
 {'name': '역삼아레나빌딩',
  'lat': 37.501087,
  'long': 127.043069,
  'code': '3762',
  'storecd': '1509',
  'addr': '서울특별시 강남구 언주로 425 (역삼동)',
  'spec': 'general'})

In [21]:
starbucks_store_infos = [ {
        'name': li.get('data-name') , # 지점명, 속성값 추출 => 요소.get('속성이름')!!!!!!!!!!!!!!!
        'lat' : float( li.get('data-lat') ) ,   # 위도 -> float()
        'long': float( li.get('data-long')) ,   # 경도 -> float() 
        'code': li.get('data-code') , # 지점코드
        'storecd': li.get('data-storecd') ,                 # 관리코드(?)
        'addr': li.p.text.strip()[:len('1522-3232')*-1] ,   # 전화번호가 모두 동일하다
        'spec': li.i.get('class')[0][len('pin_'):]  # class는 속성값으로 뽑으면 리스트로 나온다(특징)->그래서[0]해준거다, pin_ 제거    
} for li in soup.select('.quickSearchResultBoxSidoGugun > li') ]

len(starbucks_store_infos), starbucks_store_infos[-1]

(598,
 {'name': '중화역',
  'lat': 37.60170912407773,
  'long': 127.07841136432036,
  'code': '3801',
  'storecd': '1749',
  'addr': '서울특별시 중랑구 봉화산로 35 ',
  'spec': 'general'})

- 전개 순서
    - 시도 개수만큼 반복(17) => 사전 조사를 통해서 반복횟수를 결정
        - https://www.starbucks.co.kr/store/store_map.do?disp=locale 접속
            - 난수*5초 대기
        - 시도 선택
            - 난수*3초 대기
        - 전체 선택
            - 난수*초 대기
        - 추출
            - 난수*1초 대기
    - 난수를 사용해서 대기시간을 미세하게 조정해주면, 반복적으로 데이터 수집해 상대방 사이트에서 패턴이 잡히지 않는다.

In [99]:
import random
term_make = lambda x,y:random.randint(x,y)*0.1 # 0.1 ~ 0.9까지 나옴 , term_make는 여기서 함수이다.
term_make(1,5)

0.1

In [29]:
from bs4 import BeautifulSoup

import random

term_make = lambda x,y:random.randint(x,y)*0.1 # 0.1 ~ 0.9까지 나옴
driver = wd.Chrome('./chromedriver.exe')
# 모든 지점의 정보를 담는 그릇
starbucks_store_infos = list()

for sido_idx in range(1,17+1):
    # 초기 진입 사이트 접속
    driver.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')
    time.sleep(1*5+term_make(1,5)) #5초는 깔고 함수로 0.1~0.9 더해서 난수 가미

    # 시도 선택
    css_sel = f'div.loca_step1_cont > ul > li:nth-child({sido_idx}) > a'  # 이건 문자열 li:nth-child(1)여기가 1번이니까 서울이었다.
    si_do_a_tag = driver.find_element_by_css_selector(css_sel)
    si_do_a_tag.click()
    time.sleep(1*5+term_make(1,5))

    # 전체 선택, 1번 멤버가 무조건 전체이다 -> 고정
    css_sel = '#mCSB_2_container > ul > li:nth-child(1) > a'
    driver.find_element_by_css_selector(css_sel).click()
    time.sleep(1*5+term_make(1,5))

    # 정보 추출 (셀레니움 방식과 뷰티풀수프 방식)
    # soup(DOM Tree) 생성
    src = driver.page_source  # 현재 페이지의 HTML 소스 
    soup = BeautifulSoup(src,'html5lib')

    # 정보추출
    starbucks_store_local_infos = [ {
        'name': li.get('data-name') , # 지점명, 속성값 추출 => 요소.get('속성이름')
        'lat' : float( li.get('data-lat') ) ,   # 위도 -> float()
        'long': float( li.get('data-long')) ,   # 경도 -> float() 
        'code': li.get('data-code') , # 지점코드
        'storecd': li.get('data-storecd') ,                 # 관리코드(?)
        'addr': li.p.text.strip()[:len('1522-3232')*-1] ,   # 전화번호가 모두 동일하다
        'spec': li.i.get('class')[0][len('pin_'):]  # class는 속성값으로 뽑으면 리스트로 나온다(특징), pin_ 제거    
    } for li in soup.select('.quickSearchResultBoxSidoGugun > li') ] 

    # starbucks_store_infos에 starbucks_store_local_infos값 하나하나를 멤버로 추가한다. 우리는 리스트안 리스트말고 동급인 리스트안 딕셔너리 원함
    starbucks_store_infos.extend(starbucks_store_local_infos)  # 동등한 위치로 들어감
    
    if sido_idx ==2 :
        print(len(starbucks_store_infos))
        #print(starbucks_store_infos)
        break
   

1016


In [ ]:
a = []
b = [1,2]
c = [3,4]
# a.append(b) #[[1,2]]
a.extend(b) #[1,2]
a.extend(c) #[1,2,3,4]  -> 같은 급으로 들어감

In [23]:
# 브라우저 닫기
driver.close()
driver.quit()

#파이썬 파일에서 사용
# import sys
# sys.exit(0)

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(starbucks_store_infos)
df.to_excel('starbucks_store_full.xlsx', index=False)